Extracting Google Reviews Using Webscraping (selenium , BeautifulSoup)

In [1]:
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [3]:
# Link of the google map reviews of a local restaurant
url = 'https://www.google.com/maps/place/Kakinada+Subbayya+Gari+Hotel+Vijayawada/@16.5119874,80.6275706,17z/data=!4m8!3m7!1s0x3a35fb915179ff35:0x747ad0a12338024d!8m2!3d16.5119874!4d80.6301455!9m1!1b1!16s%2Fg%2F11q4jw57gk?hl=en&hl=en&entry=ttu&g_ep=EgoyMDI0MTAyMy4wIKXMDSoASAFQAw%3D%3D'
driver.get(url)

In [4]:
# To Extract more sample reviews
for _ in range(5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

In [5]:
page_content = driver.page_source

In [6]:
# Beautifulsoup is used to extract the html content of the review webpage
soup = BeautifulSoup(page_content, 'html.parser')

In [7]:
#Classes for extraction
names = soup.select('.d4r55')
stars = soup.select('.kvMYJc')
review_texts = soup.select('.MyEned')

In [9]:
final=[]
for i in range(len(names)):
    final.append({'Name': names[i].text,
                 'Ratings': stars[i].get('aria-label').replace('stars','').replace('star',''),
                  'Review' : review_texts[i].text })

In [10]:
input_reviews = json.dumps(final,indent=4)
print(input_reviews)

[
    {
        "Name": "SISTA AJAY SHARMA",
        "Ratings": "4 ",
        "Review": "I Had meals normal meals 180/- taste and service was nice and very good quality and quantityFood:\u2026 More"
    },
    {
        "Name": "Rakesh Kunchanapalli",
        "Ratings": "3 ",
        "Review": "Disappointed \ud83d\ude1e. Now this turned completely commercial. Food is average. Ambience and hygiene is not that great. And the service is very cheap and completely contrast to what subbaya gari hotel is known for. \ud83d\udc4e \u2026 More"
    },
    {
        "Name": "Kalyan Kumar",
        "Ratings": "1 ",
        "Review": "Food is not good ,not warm ,they are cold serving people are less , have to call everytime ,food taste is below average,there is a hype that's it for 280/-not at all worth More"
    },
    {
        "Name": "Teja Vattem",
        "Ratings": "5 ",
        "Review": "Few dishes are good and few are ok. But the pricing is little bit expensive. South indian meal is good. M

In [11]:
with open('reviews.json', 'w') as file:
    json.dump(final, file, indent=4)

Bullding AI Agent for Sentiment Analysis

In [12]:
import os
from dotenv import load_dotenv
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent
from langchain_google_genai import GoogleGenerativeAI
from langchain.agents.agent_types import AgentType

c:\Users\LENOVO\anaconda3\envs\review_ai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
load_dotenv()

True

In [14]:
api_key = os.getenv('GOOGLE_API_KEY')

In [15]:
llm = GoogleGenerativeAI(model="gemini-pro",api_key = api_key)

In [16]:
# This tool is used by agent to get the sentiment of the given review
def sentiment_analysis(text:str)->str:
    response = llm(

        prompt=f"What is the sentiment of the given customer Review: {text}",

        max_tokens=100

    )

    return response

In [17]:
# This tool is used by agent to give the feedback according to the sentiment and the review
def customer_feedback(review:str,sentiment:str) ->str:
    response = llm(

        prompt=f"""You are a expert in replying to the customer feedbacks professionaly. When given a customer review and its sentiment, you would reply to the review. 'Thanks for you feedback.' would be your typical start. 
        If positive or neutral: You have to respond with a sense of gratitude.
        else if negative: You have to reply by acknowledging the feedback and proper apologies.  
        The Customer Review: {review}
        Sentiment: {sentiment}""",

        max_tokens=100

    )

    return response

In [18]:
senti = StructuredTool.from_function(
    name="Get_Sentiment",
    func= sentiment_analysis,
    description="When given a customer review, this tool classifies the sentiment into classes: Positive, Neutral, and Negative"
)
feedback = StructuredTool.from_function(
    name="Feedback_replier",
    func= customer_feedback,
    description="The tool responds to a customer review professionaly based on the review and its sentiment. Input arguments: review, sentiment."
)

In [47]:
agent= initialize_agent(
    [senti, feedback],
    llm, 
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
    verbose = True)

In [56]:
feedbacks_reviews = []

In [57]:
for i in range(len(final)):
    result = agent.invoke(f"""You are a customer service reply agent. You should reply to the review given by the customer by considering the sentiment of the review: {final[i]['Review']}.""")
    feedbacks_reviews.append(result)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Get_Sentiment",
  "action_input": {
    "text": "I Had meals normal meals 180/- taste and service was nice and very good quality and quantityFood:… More."
  }
}
```
Observation: Positive
Thought:Action:
```
{
  "action": "Feedback_replier",
  "action_input": {
    "review": "I Had meals normal meals 180/- taste and service was nice and very good quality and quantityFood:… More.",
    "sentiment": "Positive"
  }
}
```
Observation: Thanks for your feedback. We're glad you enjoyed your meals and found the service to be nice. We appreciate your compliments on the taste, quality, and quantity of the food. We'll continue to strive to provide our customers with a great dining experience.
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "Thanks for your feedback. We're glad you enjoyed your meals and found the service to be nice. We appreciate your compliments on the taste, quality, and quantity of the food. 

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Action:
```
{
  "action": "Feedback_replier",
  "action_input": {
    "review": "Few dishes are good and few are ok. But the pricing is little bit expensive. South indian meal is good. More.",
    "sentiment": "Neutral"
  }
}
```
Observation: Thanks for your feedback.

We appreciate you taking the time to share your thoughts with us. We're glad to hear that you enjoyed some of our dishes, especially our South Indian meal. We value your feedback regarding pricing, and we'll take it into consideration as we review our menu options. We strive to provide our customers with great food at a reasonable price, and we'll continue to work towards finding the right balance.

Thank you again for your feedback. We hope to serve you again soon.
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "Thanks for your feedback.\n\nWe appreciate you taking the time to share your thoughts with us. We're glad to hear that you enjoyed some of our dishes, especially our South Indian meal. We va

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..



Observation: Thanks for your feedback. We sincerely apologize for the disappointing experience you had at our establishment. We deeply regret that the food was not up to par and that the manager did not respond appropriately to your requests. We take customer satisfaction very seriously and will use your feedback to improve our service and quality of food. We hope you will give us another chance to show you that we can provide a much better dining experience.
Thought:

...

Action:
```
{
  "action": "Final Answer",
  "action_input": "Thanks for you feedback. We are so happy to hear that you enjoyed your meal and that each and every single dish tasted too good. We hope to see you again soon!"
}
```

> Finished chain.


> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Get_Sentiment",
  "action_input": {
    "text": "Best veg meals in Vja. Hygienic, fresh and nutritious meals in a beautiful and spacious ambience. Thanks to Ashwak for great service. Highly recommend. More."
  }
}
```
Observation: Positive
Thought:Action:
```
{
  "action": "Feedback_replier",
  "action_input": {
    "review": "Best veg meals in Vja. Hygienic, fresh and nutritious meals in a beautiful and spacious ambience. Thanks to Ashwak for great service. Highly recommend. More.",
    "sentiment": "Positive"
  }
}
```
Observation: Thanks for you feedback. We are delighted to hear that you enjoyed your dining experience at our restaurant. Your compliments on the quality of ou

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..



Observation: Positive
Thought:Action:
```
{
  "action": "Feedback_replier",
  "action_input": {
    "review": "Food is very delicious, hygienic and tasty. Service is excellent.  All are served with love and affection. Each and every item is very nice. We have tried Tiffin,  Lunch, and buffet. Hospitality is so nice. More.",
    "sentiment": "Positive"
  }
}
```
Observation: Thanks for your feedback. We are delighted to know that you had a wonderful experience with our food and service. Your kind words about our hygiene, taste, and affectionate service are truly appreciated. We strive to provide our customers with the best possible dining experience, and it is heartening to know that we have succeeded in meeting your expectations. We are committed to maintaining our high standards and continuing to serve you with the same love and dedication. Thank you for choosing us and for your continued patronage.
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "Thanks for your 

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..



Observation: Thanks for your feedback. We're so glad you enjoyed our traditional South Indian thali! We take pride in our food and service, and we're happy to hear that we met your expectations. We hope to see you again soon!
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "Thanks for your feedback. We're so glad you enjoyed our traditional South Indian thali! We take pride in our food and service, and we're happy to hear that we met your expectations. We hope to see you again soon!"
}
```

> Finished chain.


> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Get_Sentiment",
  "action_input": {
    "text": "Ordered Andhra Veg Butta Bojanam for two people. The quantity was good enough for three people. The taste was good except the Sambar and Rasam where Bellam was added. The beerakkai pachadi and the tomato Puppu were the  best. More."
  }
}
```
Observation: Positive
Thought:Action:
```
{
  "action": "Feedback_replier",
  "action_input": {
    "revie

In [59]:
feedbacks_reviews[0]

{'input': 'You are a customer service reply agent. You should reply to the review given by the customer by considering the sentiment of the review: I Had meals normal meals 180/- taste and service was nice and very good quality and quantityFood:… More.',
 'output': "Thanks for your feedback. We're glad you enjoyed your meals and found the service to be nice. We appreciate your compliments on the taste, quality, and quantity of the food. We'll continue to strive to provide our customers with a great dining experience.",
 'intermediate_steps': [(AgentAction(tool='Get_Sentiment', tool_input={'text': 'I Had meals normal meals 180/- taste and service was nice and very good quality and quantityFood:… More.'}, log='Action:\n```\n{\n  "action": "Get_Sentiment",\n  "action_input": {\n    "text": "I Had meals normal meals 180/- taste and service was nice and very good quality and quantityFood:… More."\n  }\n}\n```'),
   'Positive'),
  (AgentAction(tool='Feedback_replier', tool_input={'review': '

In [58]:
feedbacks_reviews[0]['intermediate_steps'][0][-1]

'Positive'

In [61]:
for i in range(len(feedbacks_reviews)):
    print('--------------------------------------------------------')
    print(f'"The Customer Review": {final[i]["Review"]}')
    print(f'"The Sentiment of the Review": {feedbacks_reviews[i]["intermediate_steps"][0][-1]}')
    print(f'"The Feedback : {feedbacks_reviews[i]["output"]}"')
    print('--------------------------------------------------------')

--------------------------------------------------------
"The Customer Review": I Had meals normal meals 180/- taste and service was nice and very good quality and quantityFood:… More
"The Sentiment of the Review": Positive
"The Feedback : Thanks for your feedback. We're glad you enjoyed your meals and found the service to be nice. We appreciate your compliments on the taste, quality, and quantity of the food. We'll continue to strive to provide our customers with a great dining experience."
--------------------------------------------------------
--------------------------------------------------------
"The Customer Review": Disappointed 😞. Now this turned completely commercial. Food is average. Ambience and hygiene is not that great. And the service is very cheap and completely contrast to what subbaya gari hotel is known for. 👎 … More
"The Sentiment of the Review": Negative
"The Feedback : Thanks for your feedback.

We are truly sorry to hear that you did not have a positive experie

In [62]:
output_feedback_review = []

In [65]:
for i in range(len(feedbacks_reviews)):
    output_feedback_review.append({
        "Customer Review" : final[i]['Review'],
        "Sentiment": feedbacks_reviews[i]["intermediate_steps"][0][-1],
        "Feedback": feedbacks_reviews[i]["output"]
    })

In [66]:
with open('feedbacks_reviews.json', 'w') as file:
    json.dump(output_feedback_review, file, indent=4)